In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
import openai
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from transformers import pipeline
# import torch
# import textwrap
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import SentenceTransformerEmbeddings
# from langchain.llms import HuggingFacePipeline
# from langchain.chains import RetrievalQA

/home/nouman/anaconda3/envs/data_extraction/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from langchain import HuggingFaceHub
# from langchain import PromptTemplate, LLMChain

In [5]:
# repo_id="facebook/bart-large-cnn"
# #repo_id = "MaRiOrOsSi/t5-base-finetuned-question-answering"
# #repo_id = "MBZUAI/LaMini-T5-738M"
# #repo_id = "distilbert-base-uncased"
# llm = HuggingFaceHub(
#     repo_id=repo_id,)

In [6]:
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")

In [7]:
def data_ingestion(file):
    documents = []
    #print(file)
    loader = PDFMinerLoader(file)
    data = loader.load()
    texts = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)

    for doc in data:
        texts.append(doc.page_content)
    texts = text_splitter.split_text(texts[0])
#     texts = text_splitter.split_documents(loader.load())
    # Create embeddings here
#     embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    # docsearch = Pinecone.from_existing_index(index_name, embeddings)
    # Create and save FAISS index
#     vectorStore_openAI = FAISS.from_documents(texts, embeddings)
#     vectorStore_openAI.save_local("data/faiss/")
    return texts
    

In [8]:
file ="crime-and-punishment.pdf"
texts = data_ingestion(file)


In [9]:
# texts[5]

In [19]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = """
Please provide a concise summary of the following text, limited to 100 words. 
Avoid using phrases such as "In this passage," "Summary of this paragraph is,","The text provides" etc. 
Aim for clarity and brevity to capture the essence of the document.

TEXT:
"{text}"

SUMMARY:
"""

prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)


In [20]:
# llm_chain.run(texts[7])
print(llm_chain.run(texts[7]))



The protagonist wakes up in a small, run-down room feeling irritable and disgusted with his surroundings. He is in a state of isolation and does not want to be bothered by anyone. His landlady has stopped providing him with meals, but he has not confronted her about it. The cook, Nastasya, brings him tea and offers to buy him food. They have a conversation about his lack of work and money. Nastasya mentions that his mother sent him a letter, which excites him. He asks her to leave so he can read the letter in private. The letter reveals that his sister has been living with their mother and they are now in a better situation financially.


In [21]:
complete_summary = ""

In [22]:
from tqdm import tqdm
try:
    for i, text in enumerate(tqdm(texts)):
        summary_content = llm_chain.run(text)
        complete_summary += " " + summary_content.strip()

        # Check if 10 iterations have passed, and if so, add a paragraph break
        if (i + 1) % 5 == 0:
            complete_summary += "\n\n"  # This adds two newline characters for a paragraph break
except NotImplementedError:
    pass  # Continue the loop when NotImplementedError is raised


100%|██████████| 127/127 [08:09<00:00,  3.86s/it]


In [23]:
len(complete_summary.split())

10966

In [27]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import ParagraphStyle

# Create a PDF document
doc = SimpleDocTemplate("output.pdf", pagesize=letter)

# Create a list of flowables (elements) to add to the PDF
story = []

# Create a custom ParagraphStyle with justification and line spacing
style = ParagraphStyle(name='CustomStyle',
                       fontName='Times-Roman',
                       fontSize=11,
                       leading= 11*1.5,  # Line spacing of 1.5 times the font size
                       alignment=1)  # 1 represents 'justify' alignment

# Split the content into paragraphs
paragraphs = complete_summary.split('\n\n')
for paragraph_text in paragraphs:
    # Create a Paragraph element with the custom style
    p = Paragraph(paragraph_text.strip(), style)
    story.append(p)
    story.append(Spacer(1, 12))  # Add some space between paragraphs

# Build the PDF document
doc.build(story)
